In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

matches = pd.read_csv('WorldCupMatches.csv')
matches = matches.dropna()

matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,05 Jul 2014 - 17:00,Quarter-finals,Arena Fonte Nova,Salvador,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),51179.0,0.0,0.0,Ravshan IRMATOV (UZB),RASULOV Abduxamidullo (UZB),KOCHKAROV Bakhadyr (KGZ),255953.0,300186488.0,NED,CRC
848,2014.0,08 Jul 2014 - 17:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1.0,7.0,Germany,,58141.0,0.0,5.0,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955.0,300186474.0,BRA,GER
849,2014.0,09 Jul 2014 - 17:00,Semi-finals,Arena de Sao Paulo,Sao Paulo,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),63267.0,0.0,0.0,C�neyt �AKIR (TUR),DURAN Bahattin (TUR),ONGUN Tarik (TUR),255955.0,300186490.0,NED,ARG
850,2014.0,12 Jul 2014 - 17:00,Play-off for third place,Estadio Nacional,Brasilia,Brazil,0.0,3.0,Netherlands,,68034.0,0.0,2.0,HAIMOUDI Djamel (ALG),ACHIK Redouane (MAR),ETCHIALI Abdelhak (ALG),255957.0,300186502.0,BRA,NED


In [3]:
#replace German DR and Germany FR by Germany
#replace Soviet Union by Russia
def replace_name(df):
    if(df['Home Team Name'] in ['German DR', 'Germany FR']):
        df['Home Team Name'] = 'Germany'
    elif(df['Home Team Name'] == 'Soviet Union'):
        df['Home Team Name'] = 'Russia'
    
    if(df['Away Team Name'] in ['German DR', 'Germany FR']):
        df['Away Team Name'] = 'Germany'
    elif(df['Away Team Name'] == 'Soviet Union'):
        df['Away Team Name'] = 'Russia'
    #df['Home Team Name'] = 'Boom'
    return df
    
matches = matches.apply(replace_name, axis='columns')

matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,05 Jul 2014 - 17:00,Quarter-finals,Arena Fonte Nova,Salvador,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),51179.0,0.0,0.0,Ravshan IRMATOV (UZB),RASULOV Abduxamidullo (UZB),KOCHKAROV Bakhadyr (KGZ),255953.0,300186488.0,NED,CRC
848,2014.0,08 Jul 2014 - 17:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1.0,7.0,Germany,,58141.0,0.0,5.0,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955.0,300186474.0,BRA,GER
849,2014.0,09 Jul 2014 - 17:00,Semi-finals,Arena de Sao Paulo,Sao Paulo,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),63267.0,0.0,0.0,C�neyt �AKIR (TUR),DURAN Bahattin (TUR),ONGUN Tarik (TUR),255955.0,300186490.0,NED,ARG
850,2014.0,12 Jul 2014 - 17:00,Play-off for third place,Estadio Nacional,Brasilia,Brazil,0.0,3.0,Netherlands,,68034.0,0.0,2.0,HAIMOUDI Djamel (ALG),ACHIK Redouane (MAR),ETCHIALI Abdelhak (ALG),255957.0,300186502.0,BRA,NED


In [4]:
#drop unecessary columns
dropped_matches = matches.drop(['Datetime', 'Stadium', 'Referee', 'Assistant 1', 'Assistant 2', 'RoundID','Win conditions',
             'Home Team Initials', 'Away Team Initials', 'Half-time Home Goals', 'Half-time Away Goals',
             'Attendance', 'City', 'MatchID', 'Stage'], 1)

dropped_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name
0,1930.0,France,4.0,1.0,Mexico
1,1930.0,USA,3.0,0.0,Belgium
2,1930.0,Yugoslavia,2.0,1.0,Brazil
3,1930.0,Romania,3.0,1.0,Peru
4,1930.0,Argentina,1.0,0.0,France
...,...,...,...,...,...
847,2014.0,Netherlands,0.0,0.0,Costa Rica
848,2014.0,Brazil,1.0,7.0,Germany
849,2014.0,Netherlands,0.0,0.0,Argentina
850,2014.0,Brazil,0.0,3.0,Netherlands


In [5]:
#find who won: Home win: 1, Away win: 2, Draw: 0
dropped_matches['Winner'] = '-'

def find_winner(df):
    if(int(df['Home Team Goals']) == int(df['Away Team Goals'])):
        df['Winner'] = 0
    elif(int(df['Home Team Goals']) > int(df['Away Team Goals'])):
        df['Winner'] = 1
    else:
        df['Winner'] = 2
    return df

dropped_matches = dropped_matches.apply(find_winner, axis='columns')

dropped_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner
0,1930.0,France,4.0,1.0,Mexico,1
1,1930.0,USA,3.0,0.0,Belgium,1
2,1930.0,Yugoslavia,2.0,1.0,Brazil,1
3,1930.0,Romania,3.0,1.0,Peru,1
4,1930.0,Argentina,1.0,0.0,France,1
...,...,...,...,...,...,...
847,2014.0,Netherlands,0.0,0.0,Costa Rica,0
848,2014.0,Brazil,1.0,7.0,Germany,2
849,2014.0,Netherlands,0.0,0.0,Argentina,0
850,2014.0,Brazil,0.0,3.0,Netherlands,2


In [6]:
#create a dictionary of football team
team_name = {}
index = 0
for idx, row in matches.iterrows():
    name = row['Home Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
    name = row['Away Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
        
team_name

{'France': 0,
 'Mexico': 1,
 'USA': 2,
 'Belgium': 3,
 'Yugoslavia': 4,
 'Brazil': 5,
 'Romania': 6,
 'Peru': 7,
 'Argentina': 8,
 'Chile': 9,
 'Bolivia': 10,
 'Paraguay': 11,
 'Uruguay': 12,
 'Austria': 13,
 'Hungary': 14,
 'Egypt': 15,
 'Switzerland': 16,
 'Netherlands': 17,
 'Sweden': 18,
 'Germany': 19,
 'Spain': 20,
 'Italy': 21,
 'Czechoslovakia': 22,
 'Dutch East Indies': 23,
 'Cuba': 24,
 'Norway': 25,
 'Poland': 26,
 'England': 27,
 'Scotland': 28,
 'Turkey': 29,
 'Korea Republic': 30,
 'Russia': 31,
 'Wales': 32,
 'Northern Ireland': 33,
 'Colombia': 34,
 'Bulgaria': 35,
 'Korea DPR': 36,
 'Portugal': 37,
 'Israel': 38,
 'Morocco': 39,
 'El Salvador': 40,
 'Australia': 41,
 'Zaire': 42,
 'Haiti': 43,
 'Tunisia': 44,
 'IR Iran': 45,
 'Iran': 46,
 'Cameroon': 47,
 'New Zealand': 48,
 'Algeria': 49,
 'Honduras': 50,
 'Kuwait': 51,
 'Canada': 52,
 'Iraq': 53,
 'Denmark': 54,
 'rn">United Arab Emirates': 55,
 'Costa Rica': 56,
 'rn">Republic of Ireland': 57,
 'Saudi Arabia': 58,
 

In [7]:
#replace team name by id in team_name dictionary

def replace_team_name_by_id(df):
    df['Home Team Name'] = team_name[df['Home Team Name']]
    df['Away Team Name'] = team_name[df['Away Team Name']]
    #df['Winner'] = team_name[df['Winner']]
    return df

teamid_matches = dropped_matches.apply(replace_team_name_by_id, axis='columns')
teamid_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner
0,1930.0,0,4.0,1.0,1,1
1,1930.0,2,3.0,0.0,3,1
2,1930.0,4,2.0,1.0,5,1
3,1930.0,6,3.0,1.0,7,1
4,1930.0,8,1.0,0.0,0,1
...,...,...,...,...,...,...
847,2014.0,17,0.0,0.0,56,0
848,2014.0,5,1.0,7.0,19,2
849,2014.0,17,0.0,0.0,8,0
850,2014.0,5,0.0,3.0,17,2


In [8]:
#drop unecessary columns
teamid_matches = teamid_matches.drop(['Year', 'Home Team Goals', 'Away Team Goals'], 1)

teamid_matches

,Home Team Name,Away Team Name,Winner
0,0,1,1
1,2,3,1
2,4,5,1
3,6,7,1
4,8,0,1
...,...,...,...
847,17,56,0
848,5,19,2
849,17,8,0
850,5,17,2


In [9]:
#TRAINING STEPS


#create numpy array
X = teamid_matches.loc[:,['Home Team Name', 'Away Team Name']]
X = np.array(X).astype('float64')

#append data: simply exchange 'home team name' with 'away team name' and replace the result
_X = X.copy()

_X[:,0] = X[:,1]
_X[:,1] = X[:,0]


y = dropped_matches.loc[:,['Winner']]
y = np.array(y).astype('int')
y = np.reshape(y,(1,850))
y = y[0]


_y = y.copy()

for i in range(len(_y)):
  if(_y[i]==1):
    _y[i] = 2
  elif(_y[i] ==2):
    _y[i] = 1
    
#===========
    
X = np.concatenate((X,_X), axis= 0)

y = np.concatenate((y,_y))

In [10]:
#Train by Decision Tree
model = DecisionTreeClassifier()
model.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [11]:
def prediction(team1, team2):
  id1 = team_name[team1]
  id2 = team_name[team2]

  x = np.array([id1, id2]).astype('float64')
  x = np.reshape(x, (1,-1))
  _y = model.predict_proba(x)[0]

  text = ('Chance for '+team1+' to win '+team2+' is {}\nChance for '+team2+' to win '+team1+' is {}\nChance for '+team1+' and '+team2+' draw is {}').format(_y[1]*100,_y[2]*100,_y[0]*100)
  return _y[0], text

In [66]:
#predict match between two teams
prob1, text1 = prediction('England', 'Argentina')
print(text1)

Chance for England to win Argentina is 60.0
Chance for Argentina to win England is 20.0
Chance for England and Argentina draw is 20.0


In [64]:
#Accuracy

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

test_model = DecisionTreeClassifier()
test_model.fit(X_train, y_train)
predictions = test_model.predict(X_test)

score = accuracy_score(y_test, predictions)
score

0.4088235294117647